In [1]:
!pip install pydgraph -i https://mirrors.aliyun.com/pypi/simple

Looking in indexes: https://mirrors.aliyun.com/pypi/simple


You should consider upgrading via the 'C:\Users\admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [43]:
import pydgraph
import json

In [33]:
client_stub = pydgraph.DgraphClientStub('dgraph-alpha.deny.pub:32080')
client = pydgraph.DgraphClient(client_stub)

In [4]:
schema = """
# Define Directives and index

director.film: [uid] @reverse .
actor.film: [uid] @count .
genre: [uid] @reverse .
initial_release_date: dateTime @index(year) .
name: string @index(exact, term) @lang .
starring: [uid] .
performance.film: [uid] .
performance.character_note: string .
performance.character: [uid] .
performance.actor: [uid] .
performance.special_performance_type: [uid] .
type: [uid] .

# Define Types

type Person {
    name
    director.film
    actor.film
}

type Movie {
    name
    initial_release_date
    genre
    starring
}

type Genre {
    name
}

type Performance {
    performance.film
    performance.character
    performance.actor
}

"""

In [5]:
op = pydgraph.Operation(schema=schema)

In [6]:
client.alter(op)

In [4]:
query = """{
    runner(func: eq(Node.name, "jenkins")) {
        uid
        Node.name
        Host.ip
    }
}"""

In [5]:
res = client.txn(read_only=True).query(query)

In [6]:
res.json

b'{"runner":[{"uid":"0x2713","Node.name":"jenkins","Host.ip":"10.103.27.177"}]}'

In [18]:
mutation_sql = """
    _:x <dgraph.type> "Host" .
    _:x <Host.admin> "liuhongwei" .
    _:x <Host.ip> "10.103.27.200" .
    _:x <Node.name> "test200-test3-test-ds" .
    _:x <Node.healthy> "0" .
    _:x <Node.createAt> "2022-06-14T11:33:00" .
    _:x <Node.updateAt> "2022-06-14T11:33:00" .
"""

In [51]:
mutation_sql = """
    <0x5> * * .
"""

In [52]:
txn = client.txn()
try: 
    mutation = txn.create_mutation(del_nquads=mutation_sql)
    request = txn.create_request(mutations=[mutation], commit_now=True)
    res = txn.do_request(request)
finally:
    txn.discard()

In [68]:
mutation_json = {
    "Node.healthy": 0,
    "Node.name": "test3",
    "Node.createAt": "2022-06-14T14:14:00",
    "Node.updateAt": "2022-06-14T14:14:00",
    "Env.host": {
        "uid": "0x4"
    },
    "dgraph.type": "Env"
}

In [69]:
txn = client.txn()
try:
    mu = pydgraph.Mutation(set_json=json.dumps(mutation_json).encode('utf8'))
    request = txn.create_request(mutations=[mu], commit_now=True)
    txn.do_request(request)
finally: 
    txn.discard()

In [72]:
mutation_json = {
    "Node.name": "ZLH",
}

In [74]:
txn = client.txn()
try:
    mu = txn.mutate(set_obj=mutation_json, commit_now=True)
finally: 
    txn.discard()